In [1]:
import os 
import pandas as pd
import glob
from tqdm import tqdm
pd.options.display.max_rows = 200
import json
from scipy import stats as st
import numpy as np
import statsmodels.stats.multitest as multitest
import matplotlib.pyplot as plt
from statsmodels.sandbox.stats.multicomp import fdrcorrection0

### Load samples

In [2]:
with open("../data/samples_untreated_updated.json") as f:
    untreated=json.load(f)
with open("../data/samples_mechanisms_updated.json") as f:
    treated=json.load(f)    


### Read output from ActiveDriver

In [4]:
l,l1=[],[]
for filein in tqdm(glob.glob("/home/fran/Documents/cuppen/HPC/tunnel/cuppen/projects/P0025_PCAWG_HMF/drivers/analysis/dna-rep-ann/final-update/r-objects/non-coding-drivers/version-202211/*/*/*/*.results.activedriverwgs.tsv.gz")):
    treatment = os.path.basename(filein).split(".")[1]
    type_element = os.path.basename(filein).split(".")[2]
    ttype = os.path.basename(filein).split(".")[0]
    df = pd.read_csv(filein,sep="\t")
    df["treatment"] = treatment
    df["type_element"] = type_element
    df["cancer_type_code"] = ttype
    
    if treatment == "untreated":
        l1.append(df[(df["element_muts_obs"]>0)])
    else:
        if not(treatment in (treated[ttype])):
            print (ttype,treatment) # cancer type removed
            continue

        l.append(df[(df["fdr_element"]<0.1)&(df["element_muts_obs"]>2)])

 53%|█████▎    | 379/714 [02:12<01:47,  3.11it/s]

BLCA Alkaloid


 53%|█████▎    | 380/714 [02:12<01:44,  3.20it/s]

BLCA Alkaloid


 53%|█████▎    | 381/714 [02:12<01:42,  3.25it/s]

...

 54%|█████▎    | 383/714 [02:13<01:42,  3.23it/s]

BLCA Alkaloid


 54%|█████▍    | 384/714 [02:13<01:43,  3.18it/s]

BLCA Alkaloid


100%|██████████| 714/714 [04:04<00:00,  2.93it/s]


In [5]:
df_t = pd.concat(l).rename(columns={"cacnet_type_code":"cancer_type_code","type_element":"genomic_element"})
df_c = pd.concat(l1).rename(columns={"element_muts_obs":"MUTS_CONTROL","fdr_element":"QVALUE_CONTROL","cacnet_type_code":"cancer_type_code","type_element":"genomic_element"})

### Select drivers with fdr < 0.05, exclude drivers from Melanoma due to promoter hypermutation and non-specific genomic_elemnts 

In [6]:
comb=df_t[(df_t["fdr_element"]<0.05)&(df_t["cancer_type_code"]!="SKCM")&(df_t["genomic_element"]!="nc_elements")].merge(df_c[["id","cancer_type_code","genomic_element","MUTS_CONTROL","QVALUE_CONTROL"]],how="left").fillna({"QVALUE_CONTROL":1.0}).fillna(0.0)

### Compute enrichment in treatment groups

In [7]:
def compare_treated_untreated(row):
    # tretaed
    n_treated = row["element_muts_obs"]
    total_treated = len(treated[row["cancer_type_code"]][row["treatment"]])
    # untreated
    n_untreated = row["MUTS_CONTROL"]
    total_untreated = len(untreated[row["cancer_type_code"]]["untreated"])
    odds,pvalue=st.fisher_exact([[n_treated,total_treated-n_treated],[n_untreated,total_untreated-n_untreated]])
    if not(np.isfinite(odds)):
        odds = (n_treated / total_treated ) / ((n_untreated +0.5) / total_untreated)
    return pd.Series([np.log2(odds+0.01),pvalue])

In [8]:
comb[["odds_ratio","pvalue_fisher"]] = comb.apply(lambda row: compare_treated_untreated(row),axis=1)
# perform FDR correction
l=[]
for ttype in comb["cancer_type_code"].unique():
    q=comb[comb["cancer_type_code"]==ttype]
    q["qvalue"] = fdrcorrection0(q["pvalue_fisher"].values)[1]
    l.append(q)
comb=pd.concat(l)                                          
comb["log_qvalue"] = -np.log10(comb["qvalue"])

/home/fran/miniconda3/envs/global/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
annotated_mc = pd.read_csv("../data/SuppTable8_TEDs - Annotated TEDs.tsv",sep="\t").rename(columns={"drug":"treatment","cancertype":"cancer_type_code","gene":"id"})[["id","cancer_type_code","treatment"]]
annotated_mc["annotated"] = True
d_maxv_mc=comb[(comb["qvalue"]<0.1)].groupby(["id","cancer_type_code"]).agg({"qvalue":np.nanmin}).to_dict()["qvalue"]
comb["representative_mech"] = comb.apply(lambda row: d_maxv_mc[(row["id"],row["cancer_type_code"])] == row["qvalue"] if ((row["id"],row["cancer_type_code"]) in d_maxv_mc and row["qvalue"] < 0.1) else False,axis=1)
comb = comb.merge(annotated_mc,how="left").fillna({"annotated":False})
comb["label"] = comb.apply(lambda row: row["id"].split("_")[1]+" [" + row["genomic_element"] +  "] - " + row["treatment"] +" ("+row["cancer_type_code"]+")",axis=1)
comb["type_alt"] = "non-coding mutation"

### Save the data

In [1]:
comb[["id","treatment","cancer_type_code","genomic_element","label","type_alt","qvalue","log_qvalue","odds_ratio","element_muts_obs","MUTS_CONTROL","annotated","representative_mech"]].rename(
    columns={"id":"gene","treatment":"drug","q.value_TT_fdr":"qvalue","odds_ratio":"log_odds_ratio","element_muts_obs":"mutated_t","MUTS_CONTROL":"mutated_c"}).to_csv("../data/non_coding_results_resistance.tsv.gz",sep="\t",index=False)

NameError: name 'comb' is not defined

In [56]:
comb[(comb["qvalue"]<0.1)].to_csv("results/significant_hits_noncoding.tsv",sep="\t",index=False)